## Load data

In [66]:
import csv
import re
import numpy

class Repository(object):
    def __init__(self,fileName):
        self.fileName = fileName
    def  loadData(self,inputColumnFst,outputColumn,):
        data=[]
        columns=[]
        file = open(self.fileName)
        parsedData = csv.reader(file,delimiter=',')
       
        noLines = 0
        for line in parsedData:
            if noLines == 0:
                columns = line
            else:
                data.append(line)
            noLines += 1
                
        inputColumnIndexFst = columns.index(inputColumnFst)
        outputColumnIndex = columns.index(outputColumn)
    
        inputDataFst = []
        inputDataSnd = []
    
        outputData = []
        for j in range(len(data)):
            inputDataFst.append(data[j][inputColumnIndexFst])
            outputData.append(data[j][outputColumnIndex])
        
        file.close()
        
        return inputDataFst,outputData
    
    


In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

filename = "textEmotions.csv"
repository = Repository(filename)
inputColumnFst = 'content'
vectorizer = TfidfVectorizer(stop_words='english')

outputColumn = 'sentiment'
inputDataResultFst,outputDataResult = repository.loadData(inputColumnFst, outputColumn)

print('inputs for column '+inputColumnFst +' is: \n'+str(inputDataResultFst)+'\n')
print('outputs for column '+outputColumn +' is: \n'+str(outputDataResult)+'\n')




inputs for column content is: 
['Layin n bed with a headache  ughhhh...waitin on your call...', 'Funeral ceremony...gloomy friday...', '@dannycastillo We want to trade with someone who has Houston tickets, but no one will.', "Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends", "I should be sleep, but im not! thinking about an old friend who I want. but he's married now. damn, &amp\t he wants me 2! scandalous!", 'Hmmm. http://www.djhero.com/ is down', '@charviray Charlene my love. I miss you', "@kelcouch I'm sorry  at least it's Friday?", 'cant fall asleep', 'Choked on her retainers', 'Ugh! I have to beat this stupid song to get to the next  rude!', '@BrodyJenner if u watch the hills in london u will realise what tourture it is because were weeks and weeks late  i just watch itonlinelol', 'The storm is here and the electricity is gone', "So sleepy again and it's not even that late. I fail once again.", '@PerezHilton lady gaga tweeted about not being im

### separating training data from test data 20%-80%

In [68]:
import numpy as np 
import math

np.random.seed(5)

# generate the positions of the data that will be a part of the test data
noTestDataIndexes = math.floor( 0.2 * len(inputDataResultFst))
testDataIndexes = []
for index in range(0,noTestDataIndexes):
    testDataIndexes.append(np.random.randint(0,len(inputDataResultFst)))

inputTestFst = []
outputTest = []
inputTrainingFst = []
outputTraining = []
for i in range (0,len(inputDataResultFst)):
    if i in testDataIndexes :
        inputTestFst.append(inputDataResultFst[i])
        outputTest.append(outputDataResult[i])
    else:
        inputTrainingFst.append(inputDataResultFst[i])
        outputTraining.append(outputDataResult[i])
    
print(inputTrainingFst)
print(outputTraining)
print(inputTestFst)
print(outputTest)


def clean_text(review_text):
    import re
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    # Convert to lower case, split into individual words
    words = letters_only.lower().split()   
    return letters_only      
        
def apply_cleaning_function_to_series(X):
    print('Cleaning data')
    cleaned_X = []
    for element in X:
        cleaned_X.append(clean_text(element))
    print ('Finished')
    return cleaned_X

trainClean = apply_cleaning_function_to_series(inputTrainingFst)
testClean = apply_cleaning_function_to_series(inputTestFst)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer(analyzer = "word",
                                 tokenizer = None,
                                 preprocessor = None,
                                 stop_words = None,
                                 ngram_range = (1,2),
                                 max_features = 200) 
trainData = vectorizer.fit_transform(trainClean)
testData = vectorizer.fit_transform(testClean)
trainData = trainData.toarray()
testData = testData.toarray()
tfidf = TfidfTransformer()
featuresTrain = tfidf.fit_transform(trainData).toarray()
featuresTest =  tfidf.fit_transform(testData).toarray()

['Funeral ceremony...gloomy friday...', '@dannycastillo We want to trade with someone who has Houston tickets, but no one will.', "Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends", "I should be sleep, but im not! thinking about an old friend who I want. but he's married now. damn, &amp\t he wants me 2! scandalous!", '@charviray Charlene my love. I miss you', 'Choked on her retainers', 'Ugh! I have to beat this stupid song to get to the next  rude!', '@BrodyJenner if u watch the hills in london u will realise what tourture it is because were weeks and weeks late  i just watch itonlinelol', 'The storm is here and the electricity is gone', "So sleepy again and it's not even that late. I fail once again.", '@PerezHilton lady gaga tweeted about not being impressed by her video leaking just so you know', 'How are YOU convinced that I have always wanted you? What signals did I give off...damn I think I just lost another friend', "@raaaaaaek oh too bad! I h

**K-means implmentation**

In [69]:
def init_centroids(k, data):
    c = []
    # Generate a list of k random numbers between 0 and the length of features of dataset
    s = np.random.randint(low=1, high=len(data), size=k)
    # Check if all members of 's' are unique, if not, generate again
    while len(s) != len(set(s)):
        s = np.random.randint(low=1, high=len(data), size=k)
    # For every i in s, get ith feature from dataset, and add it to list
    for i in s:
        c.append(data[i])
    return c

def eucDistance(a, b):
    sum = 0
    for i, j in zip(a, b):
        a = (i - j) * (i - j)
        sum = sum + a
    return math.sqrt(sum)

def calcDistances(centroids, data):
    c_dist = []
    # For each centroid c, iterate through all points in data to calculate its distance from c
    for i in centroids:
        temp = []
        for j in data:
            temp.append(eucDistance(i, j))
        c_dist.append(temp)
    return c_dist

def clustering(k, distances):
    # create empty cluster list of size k
    clusters = []
    for i in range(k):
        clusters.append([])
    # start clustering data points, such that each point is clustered to nearest centroid
    for i in range(len(distances[0])):
        d = []
        for j in range(len(distances)):
            d.append(distances[j][i])
        clusters[d.index(min(d))].append(i)
    return clusters

def updateCentroids(centroids, clusters, data):
    for i in range(len(centroids)):
        if len(clusters) > 0:
            temp = []
            # Copy features of cluster members to temp list
            for j in clusters[i]:
                temp.append(list(data[j]))
            # Take mean of features of all members of cluster to get new centroid location
            sum = [0] * len(centroids[i])
            for l in temp:
                sum = [(a + b) for a, b in zip(sum, l)]
            centroids[i] = [p / len(temp) for p in sum]

    return centroids



def kMeans(k, data, epochs):
    # Initialize centroids
    centroids = init_centroids(k, data)
    # Calculate distance table
    distances = calcDistances(centroids, data)
    # Perform clustering based on above generated distance table
    clusters = clustering(k, distances)
    # Update centroid location based on above generated cluster table
    newCentroids = updateCentroids(centroids, clusters, data)

    # Repeat from step 2 till stopping criteria is met
    for i in range(epochs):
        distances = calcDistances(newCentroids, data)
        clusters = clustering(k, distances)
        newCentroids = updateCentroids(newCentroids, clusters, data)

    
    centroids = []
    # Display the final results
    for i in range(len(newCentroids)):
        print("Centroid ", i, ": ", newCentroids[i])
        centroids.append(newCentroids[i])
    return centroids

def predict(instance, centers):
    dimension = len(instance)
    centerDimension = len(centers[0])
    
    nearestCenter = None
    nearestDistance = None
    
    maxInd = -1
    minDistance = 1000
    for i in range(len(centers)):
        distance = 0
        for dim in range(1, dimension):
            dist = instance[dim] - centers[i][dim]
            distance += dist**2
        distance = math.sqrt(distance)
        if nearestDistance is None :
            nearestDistance = distance
            nearestCenter = i
        elif nearestDistance > distance:
            nearestDistance = distance
            nearestCenter = i
        if distance < minDistance :
            maxInd = i
            minDistance = distance
        print('center:',i, 'dist:',distance)
    return maxInd

**solve = train+test**

In [70]:

centers = kMeans(3,featuresTrain,11)
predicted = []
for testData in featuresTrain:
    predicted.append(predict(testData,centers))
print(predicted)








Centroid  0 :  [0.010562174698017117, 0.0, 0.006629698435706378, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03004058158648515, 0.0, 0.0, 0.0, 0.007452271271332475, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.050280158934196566, 0.015861684000830684, 0.00956109672025187, 0.009454926361461218, 0.0, 0.0, 0.0, 0.0, 0.04573571380595425, 0.027658020321281784, 0.0, 0.06454064734282561, 0.0, 0.007527299810195065, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03300350664596463, 0.015831207467673386, 0.0, 0.0, 0.029968850369267188, 0.0, 0.028288965412736473, 0.0, 0.014791531479312843, 0.006629698435706378, 0.007133182756615165, 0.0, 0.0, 0.015861684000830684, 0.010278825296077702, 0.016828270630148006, 0.0, 0.0, 0.019554594854638133, 0.0, 0.0, 0.0, 0.010768992623496364, 0.0, 0.0, 0.07163479103593023, 0.054033607155724216, 0.0, 0.018981573503492277, 0.0, 0.0, 0.024027014109782627, 0.0, 0.015894428186700522, 0.0, 0.0, 0.0, 0.009946820768648215, 0.0, 0.0, 0.010343176552724726, 0.0, 0.009454926361461218, 0.01703948322512893, 0.0, 0.005